In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
account = pd.read_csv('account.csv',sep=';')
crcard = pd.read_csv('card.csv', sep=';')
trans = pd.read_csv('trans.csv', sep=';')
client = pd.read_csv('client.csv', sep = ';')
order = pd.read_csv('order.csv', sep = ';')
loan = pd.read_csv('loan.csv', sep = ';')
disp = pd.read_csv('disp.csv', sep = ';')
district = pd.read_csv('district.csv', sep = ';')

last_date = '981231'

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
from dateutil.relativedelta import relativedelta
age = []
ccl0 = disp[disp['type'] == 'OWNER']['client_id']
ccl = client[client['client_id'].isin(ccl0)]
for i in ccl['birth_number'].values:
    xx1 = 19981231
    if (i%10000/100) >50 :
        mm = str(int(i%10000/100 - 50))
        yy = str(int(i/10000))
        dd = str(int(i%100))
        xx2 = '19' + yy + mm + dd
    else:
        xx2 = '19' + str(i)

    d1 = pd.to_datetime(str(xx1), format='%Y%m%d')
    d2 = pd.to_datetime(str(xx2), format='%Y%m%d')
    rdelta = relativedelta(d1,d2)
    
    c_id = ccl[ccl['birth_number'] == i]['client_id'].values[0]
    age.append([c_id,rdelta.years])

In [10]:
age = pd.DataFrame(age, columns = ['client_id', 'age']).reset_index(drop = True)
xage = disp[disp['type'] == 'OWNER'][['client_id','account_id']].reset_index(drop =True)

df = xage.merge(age, on = 'client_id')
#df = pd.DataFrame(df[df.index_x==df.index_y]['client_id'], columns=['client_id']).reset_index(drop=True)

In [23]:
# account ID's
accid = account['account_id'].values

# Credit card information
card_t = disp[disp['type'] == 'OWNER']
card_tt = crcard[crcard['disp_id'].isin(card_t['disp_id'])][['disp_id','type']]
card_type = card_t.merge(on='disp_id',right=card_tt)

# Insurance
insurance = trans['k_symbol'] == "POJISTNE"
ins_holder = set(trans[insurance]['account_id'])
loanstatus = loan[['account_id', 'status']].values

# Spending check
withdraw = trans['type'] == 'VYDAJ'
spenders = trans[withdraw]

customer = []
for x in accid:
    bal = trans[trans['account_id'] == x].sort_values('date').iloc[[-1]]['balance'].values[0]
    lastxtn = trans[trans['account_id'] == x].sort_values('date').iloc[[-1]]['date'].values[0]
    avgspend = np.round(spenders[spenders['account_id'] == x]['amount'].mean(),2)
    # Account, balance and HasCard status
    if x in card_type['account_id'].values:
        crd = card_type[card_type['account_id'] == x]['type_y'].values[0]
        customer.append([x,bal,crd,avgspend])
    else:
        customer.append([x,bal,-1,avgspend])
    
    xxlen = len(customer) - 1    
    
    # Loan status of account holders
    if x in loanstatus[:,0]:
        ll = loanstatus[loanstatus[:,0] == x][0][1]
        customer[xxlen] = customer[xxlen] + [ll]
    else:
        customer[xxlen] = customer[xxlen] + [-1]
        
    # Insurance holders
    if x in ins_holder:
        customer[xxlen] = customer[xxlen] + [1]
    else:
        customer[xxlen] = customer[xxlen] + [0]
    # Check for dormancy
    if lastxtn <= 981031:
        customer[xxlen] = customer[xxlen] + [1]
    else:
        customer[xxlen] = customer[xxlen] + [0]  

In [24]:
columns = ['account_id','balance', 'credit_card', 'avg_spend', 'loan_status', 'hasInsurance', 'Dormant']
customer = pd.DataFrame(customer, columns=columns)
customer.to_csv('consolidated_customer.csv', index=False)

In [25]:
# spendings vs loan
withdraw = trans['type'] == 'VYDAJ'
spenders = trans[withdraw]
spend = []
for x in accid:    
    avgspend = np.round(spenders[spenders['account_id'] == x]['amount'].mean(),2)
    if x in loanstatus:
        ln = loanstatus[loanstatus[:,0] == x][0][1]
        spend.append([x,float(avgspend),ln])       


In [26]:
# Precentage of credit card holders having loans
not_payed = customer['loan_status'] == 'B'
in_debt = customer['loan_status'] == 'D'
payed = customer['loan_status'] == 'A'
in_progress = customer['loan_status'] == 'C'
juniorcrd = customer['credit_card'] == 'junior'
classiccrd = customer['credit_card'] == 'classic'
goldcrd = customer['credit_card'] == 'gold'
loan_gold = customer[(not_payed | in_debt | payed | in_progress) & goldcrd].shape[0]
loan_junior = customer[(not_payed | in_debt | payed | in_progress) & juniorcrd].shape[0]
loan_classic = customer[(not_payed | in_debt | payed | in_progress) & classiccrd].shape[0]

gold_total = customer[goldcrd].shape[0]
classic_total = customer[classiccrd].shape[0]
junior_total = customer[juniorcrd].shape[0]
print("% of customers with loan having gold credit card:{} %".format((loan_gold/gold_total)*100))
print("% of customers with loan having junior credit card:{} %".format((loan_junior/junior_total)*100))
print("% of customers with loan having classic credit card:{} %".format((loan_classic/classic_total)*100))

% of customers with loan having gold credit card:18.181818181818183 %
% of customers with loan having junior credit card:14.482758620689657 %
% of customers with loan having classic credit card:20.18209408194234 %


In [29]:
xspend = customer['avg_spend'].values
bins = np.arange(400,25000,4000)
ind = np.digitize(xspend, bins)
xx = customer
xx['club'] = ind
result = xx.groupby(['loan_status','club']).agg({'account_id':'count'})
result = pd.DataFrame(result).reset_index()
pivotresult = result.pivot(index = 'club', columns = 'loan_status')
pivotresult

account_id                         
loan_status         -1     A     B      C     D
club                                           
1               2377.0  62.0   5.0  132.0  12.0
2                976.0  89.0  21.0  159.0  22.0
3                352.0  49.0   3.0  101.0  10.0
4                 99.0   3.0   2.0   11.0   1.0
5                 13.0   NaN   NaN    NaN   NaN
6                  1.0   NaN   NaN    NaN   NaN

In [ ]:
pivotresult.loc[1]